In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [4]:
traindata=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
testdata=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
traindata.iloc[50]

In [ ]:
traindata.info()

In [ ]:
traindata['keyword'].unique()

In [5]:

ytrain=traindata['target']
traindata.drop(['keyword','id','location','target'],axis=1,inplace=True)

In [32]:
import re
def preprocess(text,stem=False):
    text = text.lower()  # lowercase

    text = re.sub(r'[!]+', '!', text)
    text = re.sub(r'[?]+', '?', text)
    text = re.sub(r'[.]+', '.', text)
    text = re.sub(r"'", "", text)
    text = re.sub('\s+', ' ', text).strip()  # Remove and double spaces
    text = re.sub(r'&amp;?', r'and', text)  # replace & -> and
    text = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", text)  # Remove URLs
    # remove some puncts (except . ! # ?)
    text = re.sub(r'[:"$%&\*+,-/:;<=>@\\^_`{|}~]+', '', text)
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'EMOJI', text)
    
    tokens = []
    for token in text.split():
        tokens.append(token)
        """
        if token not in stop_words:
            if stem:
                tokens.append(SnowballStemmer.stem(token))
                
            else:
                tokens.append(token)"""
    return " ".join(tokens)

In [33]:
traindata.text=traindata.text.apply(lambda x :preprocess(x))

In [34]:
tokenizer=keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(traindata.text)
tokenlen=len(tokenizer.word_index)+1
word_index=tokenizer.word_index

In [13]:
'1'

'1'

In [35]:
xtrain=keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(traindata.text),maxlen=30)

In [ ]:
len(xtrain)

In [8]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2021-06-11 09:50:22--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-06-11 09:50:23--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-06-11 09:50:23--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [9]:
GLOVE_EMB = '/kaggle/working/glove.6B.300d.txt'
embeddings_index = {}

f = open(GLOVE_EMB)
for line in f:
  values = line.split()
  word = value = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' %len(embeddings_index))

Found 400000 word vectors.


In [37]:
embedding_matrix = np.zeros((tokenlen, 300))
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [16]:
'1'

'1'

In [38]:
dropoutlate=0.3
model=keras.models.Sequential()
model.add(keras.layers.Embedding(tokenlen,300,weights=[embedding_matrix],trainable=False,input_length=30))
model.add(keras.layers.SpatialDropout1D(dropoutlate))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(64,dropout=dropoutlate,recurrent_dropout=dropoutlate)))
model.add(keras.layers.Dense(1,activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),loss='binary_crossentropy',metrics=['accuracy'])


In [39]:
callback = keras.callbacks.ReduceLROnPlateau(factor=0.1,
                    min_lr = 0.01,
                    monitor = 'val_loss',
                    verbose = 1)
his=model.fit(xtrain,ytrain,batch_size=512,epochs=15,validation_split=0.2,callbacks=[callback])

Epoch 1/15
12/12 [==============================] - 11s 561ms/step - loss: 0.6768 - accuracy: 0.5577 - val_loss: 0.5874 - val_accuracy: 0.7472
Epoch 2/15
12/12 [==============================] - 6s 503ms/step - loss: 0.5718 - accuracy: 0.7328 - val_loss: 0.5302 - val_accuracy: 0.7787
Epoch 3/15
12/12 [==============================] - 6s 511ms/step - loss: 0.4971 - accuracy: 0.7761 - val_loss: 0.4471 - val_accuracy: 0.7879
Epoch 4/15
12/12 [==============================] - 6s 510ms/step - loss: 0.4615 - accuracy: 0.7948 - val_loss: 0.4418 - val_accuracy: 0.7919
Epoch 5/15
12/12 [==============================] - 6s 538ms/step - loss: 0.4419 - accuracy: 0.8018 - val_loss: 0.4211 - val_accuracy: 0.8043
Epoch 6/15
12/12 [==============================] - 6s 504ms/step - loss: 0.4371 - accuracy: 0.8047 - val_loss: 0.4234 - val_accuracy: 0.8076
Epoch 7/15
12/12 [==============================] - 6s 509ms/step - loss: 0.4325 - accuracy: 0.8102 - val_loss: 0.4230 - val_accuracy: 0.8037
Epoch

In [45]:
testdata=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
testid=testdata['id']
testdata.drop(['keyword','id','location'],axis=1,inplace=True)
testdata.text=testdata.text.apply(lambda x :preprocess(x))
tokenizer=keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(testdata.text)
xtest=keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(testdata.text),maxlen=30)

In [47]:
pre=model.predict(xtest)
print(pre[:10])
pre=list(1 if x>0.5 else 0 for x in list(pre))


[[0.08060655]
 [0.49797368]
 [0.20082992]
 [0.18990326]
 [0.12748528]
 [0.11490628]
 [0.19808981]
 [0.1257697 ]
 [0.15187395]
 [0.52492696]]


In [43]:
res=pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')
#result=pd.DataFrame((pre,testid))
res['target']=pre
res.to_csv('res.csv',index=False)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]